# Flight Price Analysis - Exploratory Data Analysis (EDA)

This notebook performs comprehensive exploratory data analysis on the Bangladesh Flight Price Dataset.

## Dataset Overview
- **Source**: Kaggle - Flight Price Dataset of Bangladesh
- **Purpose**: Understand data structure, distributions, and patterns before pipeline processing

## 1. Setup and Data Loading

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings

# Settings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

# Plot settings
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

print("Libraries imported successfully!")

In [ ]:
# Load the dataset
csv_path = 'dataset/Flight_Price_Dataset_of_Bangladesh.csv'
df = pd.read_csv(csv_path)

print(f"Dataset loaded successfully!")
print(f"Shape: {df.shape[0]:,} rows × {df.shape[1]} columns")

## 2. Basic Data Information

In [ ]:
# First few rows
print("=" * 80)
print("FIRST 5 ROWS")
print("=" * 80)
df.head()

In [ ]:
# Column names and data types
print("=" * 80)
print("COLUMN INFORMATION")
print("=" * 80)
print(f"\nTotal Columns: {len(df.columns)}\n")

for i, (col, dtype) in enumerate(zip(df.columns, df.dtypes), 1):
    print(f"{i:2}. {col:<30} → {dtype}")

In [ ]:
# Detailed info
print("=" * 80)
print("DATAFRAME INFO")
print("=" * 80)
df.info()

In [ ]:
# Memory usage
memory_mb = df.memory_usage(deep=True).sum() / (1024 * 1024)
print(f"\nTotal Memory Usage: {memory_mb:.2f} MB")

## 3. Missing Values Analysis

In [ ]:
# Check for missing values
print("=" * 80)
print("MISSING VALUES ANALYSIS")
print("=" * 80)

missing = df.isnull().sum()
missing_pct = (df.isnull().sum() / len(df) * 100).round(2)

missing_df = pd.DataFrame({
    'Column': df.columns,
    'Missing Count': missing.values,
    'Missing %': missing_pct.values
}).sort_values('Missing Count', ascending=False)

print(f"\nTotal Missing Values: {missing.sum():,}")
print(f"Columns with Missing Values: {(missing > 0).sum()}")
print("\n")
missing_df[missing_df['Missing Count'] > 0]

In [ ]:
# Visualize missing values
if missing.sum() > 0:
    fig, ax = plt.subplots(figsize=(12, 6))
    missing_cols = missing[missing > 0]
    missing_cols.plot(kind='bar', ax=ax, color='coral')
    ax.set_title('Missing Values by Column', fontsize=14, fontweight='bold')
    ax.set_xlabel('Column')
    ax.set_ylabel('Count')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()
else:
    print("✅ No missing values found in the dataset!")

## 4. Statistical Summary

In [ ]:
# Numerical columns statistics
print("=" * 80)
print("NUMERICAL COLUMNS - DESCRIPTIVE STATISTICS")
print("=" * 80)

numerical_cols = df.select_dtypes(include=[np.number]).columns.tolist()
print(f"\nNumerical Columns: {numerical_cols}\n")

df[numerical_cols].describe().T

In [ ]:
# Categorical columns statistics
print("=" * 80)
print("CATEGORICAL COLUMNS - UNIQUE VALUES")
print("=" * 80)

categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
print(f"\nCategorical Columns: {len(categorical_cols)}\n")

for col in categorical_cols:
    unique_count = df[col].nunique()
    print(f"{col}: {unique_count:,} unique values")

## 5. Airline Analysis

In [ ]:
# Airlines distribution
print("=" * 80)
print("AIRLINE DISTRIBUTION")
print("=" * 80)

airline_counts = df['Airline'].value_counts()
airline_pct = (airline_counts / len(df) * 100).round(2)

airline_df = pd.DataFrame({
    'Airline': airline_counts.index,
    'Bookings': airline_counts.values,
    'Percentage': airline_pct.values
})

print(f"\nTotal Airlines: {df['Airline'].nunique()}\n")
airline_df

In [ ]:
# Visualize airline distribution
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Bar chart
colors = plt.cm.Set3(np.linspace(0, 1, len(airline_counts)))
airline_counts.plot(kind='bar', ax=axes[0], color=colors)
axes[0].set_title('Bookings by Airline', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Airline')
axes[0].set_ylabel('Number of Bookings')
axes[0].tick_params(axis='x', rotation=45)

# Pie chart
top_n = 8
top_airlines = airline_counts.head(top_n)
others = airline_counts[top_n:].sum()
if others > 0:
    pie_data = pd.concat([top_airlines, pd.Series({'Others': others})])
else:
    pie_data = top_airlines

axes[1].pie(pie_data, labels=pie_data.index, autopct='%1.1f%%', startangle=90)
axes[1].set_title('Airline Market Share', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

## 6. Route Analysis

In [ ]:
# Top source airports
print("=" * 80)
print("TOP 10 SOURCE AIRPORTS")
print("=" * 80)

source_counts = df.groupby(['Source', 'Source Name']).size().reset_index(name='Count')
source_counts = source_counts.sort_values('Count', ascending=False).head(10)
source_counts

In [ ]:
# Top destination airports
print("=" * 80)
print("TOP 10 DESTINATION AIRPORTS")
print("=" * 80)

dest_counts = df.groupby(['Destination', 'Destination Name']).size().reset_index(name='Count')
dest_counts = dest_counts.sort_values('Count', ascending=False).head(10)
dest_counts

In [ ]:
# Top routes
print("=" * 80)
print("TOP 15 MOST POPULAR ROUTES")
print("=" * 80)

df['Route'] = df['Source'] + ' → ' + df['Destination']
route_counts = df['Route'].value_counts().head(15)

fig, ax = plt.subplots(figsize=(14, 6))
route_counts.plot(kind='barh', ax=ax, color='steelblue')
ax.set_title('Top 15 Most Popular Routes', fontsize=14, fontweight='bold')
ax.set_xlabel('Number of Bookings')
ax.set_ylabel('Route')
ax.invert_yaxis()

# Add value labels
for i, v in enumerate(route_counts.values):
    ax.text(v + 100, i, f'{v:,}', va='center')

plt.tight_layout()
plt.show()

## 7. Fare Analysis

In [ ]:
# Fare statistics
print("=" * 80)
print("FARE STATISTICS (in BDT)")
print("=" * 80)

fare_cols = ['Base Fare (BDT)', 'Tax & Surcharge (BDT)', 'Total Fare (BDT)']

for col in fare_cols:
    print(f"\n{col}:")
    print(f"  Mean:   ৳{df[col].mean():,.2f}")
    print(f"  Median: ৳{df[col].median():,.2f}")
    print(f"  Min:    ৳{df[col].min():,.2f}")
    print(f"  Max:    ৳{df[col].max():,.2f}")
    print(f"  Std:    ৳{df[col].std():,.2f}")

In [ ]:
# Fare distribution
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for i, col in enumerate(fare_cols):
    axes[i].hist(df[col], bins=50, color=['#3498db', '#2ecc71', '#e74c3c'][i], 
                 edgecolor='white', alpha=0.8)
    axes[i].set_title(f'{col} Distribution', fontsize=12, fontweight='bold')
    axes[i].set_xlabel('Amount (BDT)')
    axes[i].set_ylabel('Frequency')
    axes[i].axvline(df[col].mean(), color='red', linestyle='--', label=f'Mean: ৳{df[col].mean():,.0f}')
    axes[i].axvline(df[col].median(), color='orange', linestyle='--', label=f'Median: ৳{df[col].median():,.0f}')
    axes[i].legend()

plt.tight_layout()
plt.show()

In [ ]:
# Average fare by airline
print("=" * 80)
print("AVERAGE TOTAL FARE BY AIRLINE")
print("=" * 80)

avg_fare_airline = df.groupby('Airline')['Total Fare (BDT)'].agg(['mean', 'median', 'count'])
avg_fare_airline.columns = ['Avg Fare', 'Median Fare', 'Bookings']
avg_fare_airline = avg_fare_airline.sort_values('Avg Fare', ascending=False)
avg_fare_airline

In [ ]:
# Visualize average fare by airline
fig, ax = plt.subplots(figsize=(14, 6))

avg_fare_airline['Avg Fare'].plot(kind='bar', ax=ax, color='teal', alpha=0.8)
ax.set_title('Average Total Fare by Airline', fontsize=14, fontweight='bold')
ax.set_xlabel('Airline')
ax.set_ylabel('Average Fare (BDT)')
ax.tick_params(axis='x', rotation=45)

# Add value labels
for i, v in enumerate(avg_fare_airline['Avg Fare']):
    ax.text(i, v + 500, f'৳{v:,.0f}', ha='center', fontsize=8)

plt.tight_layout()
plt.show()

## 8. Booking Class Analysis

In [ ]:
# Booking class distribution
print("=" * 80)
print("BOOKING CLASS DISTRIBUTION")
print("=" * 80)

class_counts = df['Class'].value_counts()
class_pct = (class_counts / len(df) * 100).round(2)

class_df = pd.DataFrame({
    'Class': class_counts.index,
    'Bookings': class_counts.values,
    'Percentage': class_pct.values
})
class_df

In [ ]:
# Fare by class
print("=" * 80)
print("AVERAGE FARE BY BOOKING CLASS")
print("=" * 80)

fare_by_class = df.groupby('Class')['Total Fare (BDT)'].agg(['mean', 'median', 'min', 'max'])
fare_by_class.columns = ['Avg Fare', 'Median Fare', 'Min Fare', 'Max Fare']
fare_by_class = fare_by_class.sort_values('Avg Fare', ascending=False)
fare_by_class

In [ ]:
# Visualize fare by class
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Pie chart for class distribution
colors = ['#2ecc71', '#3498db', '#e74c3c']
axes[0].pie(class_counts, labels=class_counts.index, autopct='%1.1f%%', 
            colors=colors, startangle=90, explode=[0.02] * len(class_counts))
axes[0].set_title('Booking Class Distribution', fontsize=14, fontweight='bold')

# Box plot for fare by class
df.boxplot(column='Total Fare (BDT)', by='Class', ax=axes[1])
axes[1].set_title('Fare Distribution by Class', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Class')
axes[1].set_ylabel('Total Fare (BDT)')
plt.suptitle('')  # Remove automatic title

plt.tight_layout()
plt.show()

## 9. Seasonality Analysis

In [ ]:
# Seasonality distribution
print("=" * 80)
print("SEASONALITY DISTRIBUTION")
print("=" * 80)

season_counts = df['Seasonality'].value_counts()
season_pct = (season_counts / len(df) * 100).round(2)

season_df = pd.DataFrame({
    'Season': season_counts.index,
    'Bookings': season_counts.values,
    'Percentage': season_pct.values
})
season_df

In [ ]:
# Fare by seasonality
print("=" * 80)
print("AVERAGE FARE BY SEASONALITY")
print("=" * 80)

# Define peak seasons
peak_seasons = ['Eid ul-Fitr', 'Eid ul-Adha', 'Winter Holidays']
df['Is Peak Season'] = df['Seasonality'].isin(peak_seasons)

fare_by_season = df.groupby('Seasonality')['Total Fare (BDT)'].agg(['mean', 'median', 'count'])
fare_by_season.columns = ['Avg Fare', 'Median Fare', 'Bookings']
fare_by_season = fare_by_season.sort_values('Avg Fare', ascending=False)
fare_by_season

In [ ]:
# Peak vs Non-Peak comparison
print("=" * 80)
print("PEAK VS NON-PEAK SEASON COMPARISON")
print("=" * 80)

peak_comparison = df.groupby('Is Peak Season')['Total Fare (BDT)'].agg(['mean', 'median', 'count'])
peak_comparison.index = ['Non-Peak', 'Peak']
peak_comparison.columns = ['Avg Fare', 'Median Fare', 'Bookings']

print("\n")
print(peak_comparison)
print("\n")

# Calculate difference
peak_avg = peak_comparison.loc['Peak', 'Avg Fare']
non_peak_avg = peak_comparison.loc['Non-Peak', 'Avg Fare']
diff_pct = ((peak_avg - non_peak_avg) / non_peak_avg * 100)

print(f"Peak Season Average Fare: ৳{peak_avg:,.2f}")
print(f"Non-Peak Average Fare:    ৳{non_peak_avg:,.2f}")
print(f"Difference:               ৳{peak_avg - non_peak_avg:,.2f} ({diff_pct:+.1f}%)")

In [ ]:
# Visualize seasonality
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Bar chart for average fare by season
colors = ['#e74c3c' if s in peak_seasons else '#3498db' for s in fare_by_season.index]
fare_by_season['Avg Fare'].plot(kind='bar', ax=axes[0], color=colors)
axes[0].set_title('Average Fare by Seasonality', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Season')
axes[0].set_ylabel('Average Fare (BDT)')
axes[0].tick_params(axis='x', rotation=45)
axes[0].axhline(df['Total Fare (BDT)'].mean(), color='green', linestyle='--', 
                label=f'Overall Mean: ৳{df["Total Fare (BDT)"].mean():,.0f}')
axes[0].legend()

# Box plot for peak vs non-peak
df.boxplot(column='Total Fare (BDT)', by='Is Peak Season', ax=axes[1])
axes[1].set_xticklabels(['Non-Peak', 'Peak'])
axes[1].set_title('Fare Distribution: Peak vs Non-Peak', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Season Type')
axes[1].set_ylabel('Total Fare (BDT)')
plt.suptitle('')

plt.tight_layout()
plt.show()

## 10. Duration and Stopovers Analysis

In [ ]:
# Duration statistics
print("=" * 80)
print("FLIGHT DURATION STATISTICS")
print("=" * 80)

print(f"\nDuration (hours):")
print(f"  Mean:   {df['Duration (hrs)'].mean():.2f} hrs")
print(f"  Median: {df['Duration (hrs)'].median():.2f} hrs")
print(f"  Min:    {df['Duration (hrs)'].min():.2f} hrs")
print(f"  Max:    {df['Duration (hrs)'].max():.2f} hrs")

In [ ]:
# Stopovers distribution
print("=" * 80)
print("STOPOVERS DISTRIBUTION")
print("=" * 80)

stopover_counts = df['Stopovers'].value_counts()
stopover_pct = (stopover_counts / len(df) * 100).round(2)

stopover_df = pd.DataFrame({
    'Stopovers': stopover_counts.index,
    'Bookings': stopover_counts.values,
    'Percentage': stopover_pct.values
})
stopover_df

In [ ]:
# Duration distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram of duration
axes[0].hist(df['Duration (hrs)'], bins=50, color='purple', edgecolor='white', alpha=0.8)
axes[0].set_title('Flight Duration Distribution', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Duration (hours)')
axes[0].set_ylabel('Frequency')
axes[0].axvline(df['Duration (hrs)'].mean(), color='red', linestyle='--', 
                label=f'Mean: {df["Duration (hrs)"].mean():.1f} hrs')
axes[0].legend()

# Pie chart for stopovers
colors = ['#2ecc71', '#e74c3c', '#f39c12']
axes[1].pie(stopover_counts, labels=stopover_counts.index, autopct='%1.1f%%', 
            colors=colors[:len(stopover_counts)], startangle=90)
axes[1].set_title('Stopovers Distribution', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

## 11. Booking Source Analysis

In [ ]:
# Booking source distribution
print("=" * 80)
print("BOOKING SOURCE DISTRIBUTION")
print("=" * 80)

source_counts = df['Booking Source'].value_counts()
source_pct = (source_counts / len(df) * 100).round(2)

source_df = pd.DataFrame({
    'Source': source_counts.index,
    'Bookings': source_counts.values,
    'Percentage': source_pct.values
})
source_df

In [ ]:
# Fare by booking source
print("=" * 80)
print("AVERAGE FARE BY BOOKING SOURCE")
print("=" * 80)

fare_by_source = df.groupby('Booking Source')['Total Fare (BDT)'].agg(['mean', 'median', 'count'])
fare_by_source.columns = ['Avg Fare', 'Median Fare', 'Bookings']
fare_by_source = fare_by_source.sort_values('Avg Fare', ascending=False)
fare_by_source

## 12. Days Before Departure Analysis

In [ ]:
# Days before departure statistics
print("=" * 80)
print("DAYS BEFORE DEPARTURE STATISTICS")
print("=" * 80)

print(f"\nDays Before Departure:")
print(f"  Mean:   {df['Days Before Departure'].mean():.1f} days")
print(f"  Median: {df['Days Before Departure'].median():.1f} days")
print(f"  Min:    {df['Days Before Departure'].min()} days")
print(f"  Max:    {df['Days Before Departure'].max()} days")

In [ ]:
# Create booking lead time categories
def categorize_lead_time(days):
    if days <= 7:
        return 'Last Minute (0-7 days)'
    elif days <= 14:
        return 'Short Notice (8-14 days)'
    elif days <= 30:
        return 'Standard (15-30 days)'
    elif days <= 60:
        return 'Advance (31-60 days)'
    else:
        return 'Early Bird (60+ days)'

df['Lead Time Category'] = df['Days Before Departure'].apply(categorize_lead_time)

# Lead time distribution
lead_counts = df['Lead Time Category'].value_counts()
lead_order = ['Last Minute (0-7 days)', 'Short Notice (8-14 days)', 'Standard (15-30 days)', 
              'Advance (31-60 days)', 'Early Bird (60+ days)']
lead_counts = lead_counts.reindex(lead_order)

print("\nBooking Lead Time Distribution:")
for cat, count in lead_counts.items():
    pct = count / len(df) * 100
    print(f"  {cat}: {count:,} ({pct:.1f}%)")

In [ ]:
# Fare by lead time category
print("=" * 80)
print("AVERAGE FARE BY BOOKING LEAD TIME")
print("=" * 80)

fare_by_lead = df.groupby('Lead Time Category')['Total Fare (BDT)'].agg(['mean', 'median', 'count'])
fare_by_lead.columns = ['Avg Fare', 'Median Fare', 'Bookings']
fare_by_lead = fare_by_lead.reindex(lead_order)
fare_by_lead

In [ ]:
# Visualize lead time
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Distribution of days before departure
axes[0].hist(df['Days Before Departure'], bins=50, color='teal', edgecolor='white', alpha=0.8)
axes[0].set_title('Days Before Departure Distribution', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Days')
axes[0].set_ylabel('Frequency')
axes[0].axvline(df['Days Before Departure'].mean(), color='red', linestyle='--', 
                label=f'Mean: {df["Days Before Departure"].mean():.0f} days')
axes[0].legend()

# Fare by lead time category
fare_by_lead['Avg Fare'].plot(kind='bar', ax=axes[1], color='coral')
axes[1].set_title('Average Fare by Booking Lead Time', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Lead Time Category')
axes[1].set_ylabel('Average Fare (BDT)')
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## 13. Correlation Analysis

In [ ]:
# Correlation between numerical variables
print("=" * 80)
print("CORRELATION MATRIX (Numerical Variables)")
print("=" * 80)

numerical_cols = ['Duration (hrs)', 'Base Fare (BDT)', 'Tax & Surcharge (BDT)', 
                  'Total Fare (BDT)', 'Days Before Departure']

correlation = df[numerical_cols].corr()
correlation

In [ ]:
# Correlation heatmap
fig, ax = plt.subplots(figsize=(10, 8))

sns.heatmap(correlation, annot=True, cmap='RdYlBu_r', center=0, 
            square=True, linewidths=0.5, fmt='.2f', ax=ax)
ax.set_title('Correlation Heatmap', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

## 14. Summary Statistics

In [ ]:
# Final summary
print("=" * 80)
print("DATASET SUMMARY")
print("=" * 80)

print(f"""
📊 DATASET OVERVIEW
   Total Records:      {len(df):,}
   Total Columns:      {len(df.columns)}
   Missing Values:     {df.isnull().sum().sum():,}

✈️ AIRLINES
   Unique Airlines:    {df['Airline'].nunique()}
   Most Popular:       {df['Airline'].value_counts().index[0]}

🛫 ROUTES
   Source Airports:    {df['Source'].nunique()}
   Dest. Airports:     {df['Destination'].nunique()}
   Unique Routes:      {df['Route'].nunique()}

💰 FARES (BDT)
   Average Fare:       ৳{df['Total Fare (BDT)'].mean():,.2f}
   Median Fare:        ৳{df['Total Fare (BDT)'].median():,.2f}
   Min Fare:           ৳{df['Total Fare (BDT)'].min():,.2f}
   Max Fare:           ৳{df['Total Fare (BDT)'].max():,.2f}

🎫 BOOKING CLASSES
   Economy:            {(df['Class'] == 'Economy').sum():,} ({(df['Class'] == 'Economy').sum()/len(df)*100:.1f}%)
   Business:           {(df['Class'] == 'Business').sum():,} ({(df['Class'] == 'Business').sum()/len(df)*100:.1f}%)
   First Class:        {(df['Class'] == 'First Class').sum():,} ({(df['Class'] == 'First Class').sum()/len(df)*100:.1f}%)

📅 SEASONALITY
   Peak Season:        {df['Is Peak Season'].sum():,} ({df['Is Peak Season'].sum()/len(df)*100:.1f}%)
   Non-Peak:           {(~df['Is Peak Season']).sum():,} ({(~df['Is Peak Season']).sum()/len(df)*100:.1f}%)
   Peak Fare Premium:  {((df[df['Is Peak Season']]['Total Fare (BDT)'].mean() / df[~df['Is Peak Season']]['Total Fare (BDT)'].mean() - 1) * 100):.1f}%

⏱️ FLIGHT DURATION
   Avg Duration:       {df['Duration (hrs)'].mean():.2f} hours
   Direct Flights:     {(df['Stopovers'] == 'Direct').sum():,} ({(df['Stopovers'] == 'Direct').sum()/len(df)*100:.1f}%)
""")

---

## Conclusion

This EDA provides a comprehensive overview of the Bangladesh Flight Price Dataset:

1. **Data Quality**: The dataset is clean with minimal missing values
2. **Airlines**: Multiple airlines with varying market shares
3. **Routes**: Diverse source-destination combinations
4. **Fares**: Wide range with notable differences by class and season
5. **Seasonality**: Clear fare premiums during peak seasons
6. **Booking Patterns**: Various lead times with different average fares

This analysis helps validate the data before running the Airflow pipeline.